In [ ]:
import gradio as gr
from diffusers import StableDiffusionPipeline , StableDiffusionXLPipeline
import torch
import time

**Load pre-trained Stable Diffusion models**

In [ ]:
pipe_1_5 = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda")

pipe_sdxl =  StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

text_encoder/model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

text_encoder_2/model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/5.14G [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.fp16.saf(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

**Image generation function**

In [ ]:
# History of generated images
history_images = []

def generate(prompt, negative_prompt, model_name, style, cfg_scale, size):
    start = time.time()

    full_prompt = f"{prompt}, {style}" if style else prompt
    width, height = map(int, size.split("x"))

    pipe = pipe_1_5 if model_name == "SD 1.5" else pipe_sdxl
    result = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        guidance_scale=cfg_scale,
        height=height,
        width=width
    )
    image = result.images[0]
    duration = round(time.time() - start, 2)

    history_images.append(image)

    return image, f"Generated in {duration} seconds", history_images


**Gradio UI**

In [ ]:
with gr.Blocks(css=".gr-button {background-color: #6366f1 !important; color: white}") as demo:
    gr.Markdown(
        """
        # Prompt-to-Image Generator
        Generate high-quality images using Stable Diffusion 1.5 or SDXL.
        """
    )
    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(label="Prompt", placeholder="Describe the image...")
            negative_prompt = gr.Textbox(label="Negative Prompt", placeholder="What do you NOT want in the image?")
            style = gr.Dropdown(label="Style / Theme", choices=["", "cyberpunk", "watercolor", "anime", "3D render", "photo-realistic"] , value="watercolor")

            # Choose between two model versions
            model_name = gr.Radio(choices=["SD 1.5", "SDXL"], label="Model")

            # CFG scale slider: how strictly the model should follow the prompt
            cfg = gr.Slider(1, 15, value=7, label="CFG Scale", info="How strongly the model follows your prompt")

            # Select output resolution
            size = gr.Dropdown(
                ["512x512", "768x512", "512x768"],
                value="512x512",
                label="Image Size"
                )
            generate_btn = gr.Button("Generate")

            # Display generation time
            status = gr.Text(label="Generation Time")

        with gr.Column(scale=2):
            output = gr.Image(label="Generated Image")
            gallery = gr.Gallery(label="History", show_label=True, columns=3, height=300)

    generate_btn.click(
        fn=generate,
        inputs=[prompt, negative_prompt, model_name, style, cfg, size],
        outputs=[output, status, gallery]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b5a1285f6b4faf2ee9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
